# Résolution de Sudoku avec OR-Tools

## Introduction

Google OR-Tools est une suite de logiciels d'optimisation développée par Google. Elle permet de résoudre des problèmes complexes d'optimisation combinatoire tels que la satisfaction de contraintes (CSP), la programmation linéaire (LP), la programmation linéaire mixte (MIP), et bien plus. Dans ce projet, nous nous concentrerons sur la résolution de Sudokus en utilisant différentes techniques fournies par OR-Tools.

### Types de Solveurs

1. **Solveur de Satisfaction de Contraintes (CSP)** :
   Ce solveur utilise des techniques de propagation de contraintes et de recherche pour trouver des solutions qui satisfont toutes les contraintes spécifiées.

2. **Solveur de Programmation Linéaire Mixte (MIP)** :
   Ce solveur combine la programmation linéaire avec des variables entières pour résoudre des problèmes d'optimisation. Il est particulièrement efficace pour les problèmes où certaines variables doivent être entières.

3. **Solveur de Satisfaction de Contraintes SAT (CP-SAT)** :
   Ce solveur est basé sur un noyau SAT, utilisant des techniques avancées pour résoudre les problèmes de satisfaction de contraintes en convertissant le problème en une forme satisfiable.


Nous allons explorer ces solveurs pour résoudre des grilles de Sudoku et comparer leurs performances.

## Installation de OR-Tools

Pour utiliser OR-Tools avec C#, nous devons d'abord installer la bibliothèque. Vous pouvez suivre les instructions sur le site officiel de [Google OR-Tools](https://developers.google.com/optimization).

### Installation de OR-Tools

On appelle le package Nuget correspondant. Nous utiliserons égalemnet la bibliothèque Plotly pour afficher les résultats.

In [14]:
#r "nuget: Google.OrTools"
#r "nuget: XPlot.Plotly.Interactive"

Installed Packages Google.OrTools, 9.10.4067 XPlot.Plotly.Interactive, 4.1.0

## Importation des Classes de Base

Nous allons importer les classes de base définies dans le notebook précédent, fournissant notamment la représentation, le chargement et l'affichage de Sudokus, et l'infrastructure de résolution.


In [15]:
#!import Sudoku-0-Environment.ipynb

# Notebook 0: Classes de Base pour la Résolution de Sudoku

Ce notebook contient les classes de base nécessaires pour la manipulation et la résolution des grilles de Sudoku. Il sera importé dans les autres notebooks pour réutiliser ces classes.

## Importation des Bibliothèques Nécessaires

Nous commençons par importer les bibliothèques nécessaires.


## Définition de la classe SudokuGrid

Nous définissons ici la classe SudokuGrid qui représente une grille de Sudoku et fournit des méthodes pour manipuler et afficher les grilles.


## Définition de l'interface ISudokuSolver

Nous définissons ici l'interface ISudokuSolver qui sera implémentée par les différentes stratégies de résolution de Sudoku.


## Définition de la classe SudokuHelper

Nous ajoutons ici la classe SudokuHelper qui contient des méthodes utilitaires pour charger et afficher des grilles de Sudoku.


## Affichage des Puzzles de chaque Difficulté

Nous allons charger et afficher un puzzle de chaque niveau de difficulté : Facile, Moyen et Difficile.


Puzzle Facile:
-------------------------------
| 9     2 |       5 | 4     3 | 
| 1       |    6  3 |    2  5 | 
| 5     8 | 4     7 |    6    | 
-------------------------------
|    2  6 | 3     9 |       1 | 
|    5  7 |    1    | 2  9    | 
|    9    | 6  7    | 5  3    | 
-------------------------------
| 2  4    | 5  3    | 6       | 
| 7     5 | 2       | 3     4 | 
|    8    |    4  1 | 9  5    | 
-------------------------------

Puzzle Moyen:
-------------------------------
| 8  5    |       2 | 4       | 
| 7  2    |         |       9 | 
|       4 |         |         | 
-------------------------------
|         | 1     7 |       2 | 
| 3     5 |         | 9       | 
|    4    |         |         | 
-------------------------------
|         |    8    |    7    | 
|    1  7 |         |         | 
|         |    3  6 |    4    | 
-------------------------------

Puzzle Difficile:
-------------------------------
| 4       |         | 8     5 | 
|    3    |         |         | 
|         | 7       |         | 
-------------------------------
|    2    |         |    6    | 
|         |    8    | 4       | 
|         |    1    |         | 
-------------------------------
|         | 6     3 |    7    | 
| 5       | 2       |         | 
| 1     4 |         |         | 
-------------------------------

## Implémentations des Solveurs

### Solveur par contraintes classique (CpSolver)

L'algorithme de satisfaction de contraintes (CSP) utilise les contraintes pour réduire l'espace de recherche et trouver une solution qui satisfait toutes les contraintes du problème. Nous utilisons `CpSolver` d'OR-Tools pour cette implémentation.


In [16]:
using Google.OrTools.ConstraintSolver;
using SimpleCPSolver = Google.OrTools.ConstraintSolver.Solver;
using SimpleConstraint = Google.OrTools.ConstraintSolver.Constraint;
using IntVar = Google.OrTools.ConstraintSolver.IntVar;
using System.Linq;
using System.Text;

public class OrToolsCPSolver : ISudokuSolver
{
    private const int GridSize = 9;
    private const int RegionSize = 3;
    public int VariableSelectionStrategy { get; set; } = SimpleCPSolver.CHOOSE_FIRST_UNBOUND;
    public int ValueSelectionStrategy { get; set; } = SimpleCPSolver.ASSIGN_MIN_VALUE;
    public SudokuGrid Solve(SudokuGrid s)
    {
        int[,] grid = s.Cells;
        SimpleCPSolver solver = new SimpleCPSolver("CpSimple");
        IntVar[,] matrix = CreateConstraints(solver, grid);
        // Parametrize DecisionBuilder
        DecisionBuilder db = solver.MakePhase(matrix.Flatten(), VariableSelectionStrategy, ValueSelectionStrategy);
        solver.NewSearch(db);
        while (solver.NextSolution())
        {
            string solvedString = BuildSolvedString(matrix);
            solver.EndSearch();
            return SudokuGrid.ReadSudoku(solvedString);
        }
        throw new Exception("Unfeasible Sudoku");
    }
    private static IntVar[,] CreateConstraints(SimpleCPSolver solver, int[,] grid)
    {
        IntVar[,] matrix = solver.MakeIntVarMatrix(GridSize, GridSize, 1, 9, "matrix");
        for (int i = 0; i < GridSize; i++)
        {
            for (int j = 0; j < GridSize; j++)
            {
                if (grid[i, j] != 0)
                {
                    solver.Add(matrix[i, j] == grid[i, j]);
                }
            }
        }
        for (int i = 0; i < GridSize; i++)
        {
            solver.Add(solver.MakeAllDifferent((from j in Enumerable.Range(0, GridSize) select matrix[i, j]).ToArray()));
            solver.Add(solver.MakeAllDifferent((from j in Enumerable.Range(0, GridSize) select matrix[j, i]).ToArray()));
        }
        for (int row = 0; row < GridSize; row += RegionSize)
        {
            for (int col = 0; col < GridSize; col += RegionSize)
            {
                IntVar[] regionVars = new IntVar[RegionSize * RegionSize];
                for (int r = 0; r < RegionSize; r++)
                {
                    for (int c = 0; c < RegionSize; c++)
                    {
                        regionVars[r * RegionSize + c] = matrix[row + r, col + c];
                    }
                }
                solver.Add(solver.MakeAllDifferent(regionVars));
            }
        }
        return matrix;
    }
    private static string BuildSolvedString(IntVar[,] matrix)
    {
        StringBuilder sb = new StringBuilder();
        for (int i = 0; i < GridSize; i++)
        {
            for (int j = 0; j < GridSize; j++)
            {
                sb.Append((int)matrix[i, j].Value());
            }
        }
        return sb.ToString();
    }
}


### Solveur par contraintes SAT

Le nouveau solver par contrainte d'OR-Tools est intégré au solveur SAT


In [17]:
using Google.OrTools.Sat;
using SatIntVar = Google.OrTools.Sat.IntVar;
using System;

public class OrToolsSatSolver : ISudokuSolver
{
    private const int Dimension = 9;
    private const int SubGrid = 3;
    private readonly CpSolver _solver = new CpSolver();
    
    public SudokuGrid Solve(SudokuGrid inputGrid)
    {
        (CpModel model, SatIntVar[,] grid) = CreateModel(inputGrid);
        CpSolverStatus status = _solver.Solve(model);
        if (status is CpSolverStatus.Feasible or CpSolverStatus.Optimal)
        {
            return MakeSolution(_solver, grid);
        }
        else
        {
            throw new InvalidOperationException("Sudoku grid has no solution.");
        }
    }
    private SudokuGrid MakeSolution(CpSolver solver, SatIntVar[,] grid)
    {
        SudokuGrid result = new SudokuGrid();
        for (int i = 0; i < Dimension; i++)
        {
            for (int j = 0; j < Dimension; j++)
            {
                result.Cells[i, j] = (int)solver.Value(grid[i, j]);
            }
        }
        return result;
    }
    private (CpModel model, SatIntVar[,]) CreateModel(SudokuGrid sudokuGrid)
    {
        CpModel model = new CpModel();
        SatIntVar[,] grid = new SatIntVar[Dimension, Dimension];
        CreateVariables(model, grid, sudokuGrid);
        AddConstraints(model, grid);
        return (model, grid);
    }
    private void AddConstraints(CpModel model, SatIntVar[,] grid)
    {
        for (int i = 0; i < Dimension; i++)
        {
            AddRowConstraint(model, grid, i);
            AddColumnConstraint(model, grid, i);
        }
        for (int i = 0; i < Dimension; i += SubGrid)
        {
            for (int j = 0; j < Dimension; j += SubGrid)
            {
                AddCellConstraint(model, grid, i, j);
            }
        }
    }
    private void AddCellConstraint(CpModel model, SatIntVar[,] grid, int row, int col)
    {
        SatIntVar[] cellVariables = new SatIntVar[SubGrid * SubGrid];
        for (int i = 0; i < SubGrid; i++)
        {
            for (int j = 0; j < SubGrid; j++)
            {
                cellVariables[i * SubGrid + j] = grid[row + i, col + j];
            }
        }
        model.AddAllDifferent(cellVariables);
    }
    private void AddColumnConstraint(CpModel model, SatIntVar[,] grid, int col)
    {
        SatIntVar[] colVariables = new SatIntVar[Dimension];
        for (int row = 0; row < Dimension; row++)
        {
            colVariables[row] = grid[row, col];
        }
        model.AddAllDifferent(colVariables);
    }
    private void AddRowConstraint(CpModel model, SatIntVar[,] grid, int row)
    {
        SatIntVar[] rowVariables = new SatIntVar[Dimension];
        for (int col = 0; col < Dimension; col++)
        {
            rowVariables[col] = grid[row, col];
        }
        model.AddAllDifferent(rowVariables);
    }
    private void CreateVariables(CpModel model, SatIntVar[,] grid, SudokuGrid sudokuGrid)
    {
        for (int i = 0; i < Dimension; i++)
        {
            for (int j = 0; j < Dimension; j++)
            {
                int value = sudokuGrid.Cells[i, j];
                grid[i, j] = model.NewIntVar(value == 0 ? 1 : value, value == 0 ? Dimension : value, $"Cell({i},{j})");
            }
        }
    }
}


### Solveur MIP (Mixed-Integer Programming)

L'algorithme de programmation linéaire mixte (MIP) combine la programmation linéaire et les variables entières pour résoudre des problèmes d'optimisation. Nous utilisons `Mixed-Integer Programming` d'OR-Tools pour cette implémentation.


In [18]:
using Google.OrTools.LinearSolver;
using LinearSolver = Google.OrTools.LinearSolver.Solver;
using LinearExpr = Google.OrTools.LinearSolver.LinearExpr;

public class OrToolsMIPSolver : ISudokuSolver
{
    private const int GridSize = 9;

    public SudokuGrid Solve(SudokuGrid s)
    {
        LinearSolver solver = LinearSolver.CreateSolver("SCIP");
        if (solver == null)
        {
            throw new InvalidOperationException("Solver initialization failed.");
        }

        Variable[,,] cells = new Variable[GridSize, GridSize, GridSize];
        InitializeVariables(s, solver, cells);

        AddConstraints(solver, cells);

        if (solver.Solve() != LinearSolver.ResultStatus.OPTIMAL)
        {
            throw new Exception("No solution found.");
        }

        return ExtractSolution(s, cells);
    }

    private void InitializeVariables(SudokuGrid s, LinearSolver solver, Variable[,,] cells)
    {
        for (int i = 0; i < GridSize; i++)
        {
            for (int j = 0; j < GridSize; j++)
            {
                for (int k = 0; k < GridSize; k++)
                {
                    cells[i, j, k] = solver.MakeIntVar(0, 1, $"Cell({i},{j},{k})");
                }
                if (s.Cells[i, j] != 0)
                {
                    solver.Add(cells[i, j, s.Cells[i, j] - 1] == 1);
                }
            }
        }
    }

    private void AddConstraints(LinearSolver solver, Variable[,,] cells)
    {
        for (int i = 0; i < GridSize; i++)
        {
            for (int j = 0; j < GridSize; j++)
            {
                solver.Add(Sum(solver, Enumerable.Range(0, GridSize).Select(k => cells[i, j, k])) == 1);
            }
        }

        for (int k = 0; k < GridSize; k++)
        {
            for (int i = 0; i < GridSize; i++)
            {
                solver.Add(Sum(solver, Enumerable.Range(0, GridSize).Select(j => cells[i, j, k])) == 1);
                solver.Add(Sum(solver, Enumerable.Range(0, GridSize).Select(j => cells[j, i, k])) == 1);
            }
        }

        for (int k = 0; k < GridSize; k++)
        {
            for (int i = 0; i < GridSize; i += 3)
            {
                for (int j = 0; j < GridSize; j += 3)
                {
                    solver.Add(Sum(solver, Enumerable.Range(0, 3).SelectMany(row => Enumerable.Range(0, 3).Select(col => cells[i + row, j + col, k]))) == 1);
                }
            }
        }
    }

    private LinearExpr Sum(LinearSolver solver, IEnumerable<Variable> vars)
    {
        LinearExpr sum = new();
        foreach (var v in vars)
        {
            sum += v;
        }
        return sum;
    }

    private SudokuGrid ExtractSolution(SudokuGrid s, Variable[,,] cells)
    {
        SudokuGrid solution = new SudokuGrid();
        for (int i = 0; i < GridSize; i++)
        {
            for (int j = 0; j < GridSize; j++)
            {
                for (int k = 0; k < GridSize; k++)
                {
                    if (cells[i, j, k].SolutionValue() == 1)
                    {
                        solution.Cells[i, j] = k + 1;
                    }
                }
            }
        }
        return solution;
    }
}



## Comparaison des Performances des Solveurs

Nous allons tester nos solveurs implémentés sur des grilles de Sudoku de différentes difficultés : Facile, Moyen et Difficile. Nous mesurerons également le temps de résolution et vérifierons la validité des solutions trouvées. 

Les solveurs testés sont :
- **Solveur de Satisfaction de Contraintes (CSP) avec différents DecisionBuilder**
  - Choix par défaut
  - Choix simple
  - Choix taille minimale
- **Solveur de Programmation Linéaire Mixte (MIP)**
- **Solveur de Satisfaction de Contraintes SAT (CP-SAT)**

Le test est effectué sur un ensemble de 10 Sudokus pour chaque difficulté et chaque solveur. Les résultats incluent le nombre de Sudokus résolus et le temps moyen de résolution.



In [19]:
using XPlot.Plotly;
using System.Linq;
using Trace = XPlot.Plotly.Trace;

var cpSolverDefault = new OrToolsCPSolver();
var cpSolverSimple = new OrToolsCPSolver
{
    VariableSelectionStrategy = SimpleCPSolver.INT_VAR_SIMPLE,
    ValueSelectionStrategy = SimpleCPSolver.INT_VALUE_SIMPLE
};
var cpSolverMinSize = new OrToolsCPSolver
{
    VariableSelectionStrategy = SimpleCPSolver.CHOOSE_MIN_SIZE_LOWEST_MIN,
    ValueSelectionStrategy = SimpleCPSolver.ASSIGN_CENTER_VALUE
};
var mipSolver = new OrToolsMIPSolver();
var satSolver = new OrToolsSatSolver();

List<(string SolverName, string Difficulty, double Time, int SolvedCount)> results = new();

void TestSolver(ISudokuSolver solver, SudokuDifficulty difficulty, string solverName, int numberOfSudokus)
{
    var sudokus = SudokuHelper.GetSudokus(difficulty).Take(numberOfSudokus).ToList();
    Stopwatch stopwatch = new Stopwatch();
    int solvedCount = 0;

    stopwatch.Start();
    foreach (var sudoku in sudokus)
    {
            SudokuGrid solved = solver.Solve(sudoku);
    }
    stopwatch.Stop();

    double totalTime = stopwatch.Elapsed.TotalMilliseconds;
    results.Add((solverName, difficulty.ToString(), totalTime, solvedCount));
}

int numberOfSudokus = 50;
TestSolver(cpSolverDefault, SudokuDifficulty.Easy, "CP Solver Default", numberOfSudokus);
TestSolver(cpSolverSimple, SudokuDifficulty.Easy, "CP Solver Simple", numberOfSudokus);
TestSolver(cpSolverMinSize, SudokuDifficulty.Easy, "CP Solver Min Size", numberOfSudokus);
TestSolver(mipSolver, SudokuDifficulty.Easy, "MIP Solver", numberOfSudokus);
TestSolver(satSolver, SudokuDifficulty.Easy, "SAT Solver", numberOfSudokus);
TestSolver(cpSolverDefault, SudokuDifficulty.Medium, "CP Solver Default", numberOfSudokus);
TestSolver(cpSolverSimple, SudokuDifficulty.Medium, "CP Solver Simple", numberOfSudokus);
TestSolver(cpSolverMinSize, SudokuDifficulty.Medium, "CP Solver Min Size", numberOfSudokus);
TestSolver(mipSolver, SudokuDifficulty.Medium, "MIP Solver", numberOfSudokus);
TestSolver(satSolver, SudokuDifficulty.Medium, "SAT Solver", numberOfSudokus);
TestSolver(cpSolverDefault, SudokuDifficulty.Hard, "CP Solver Default", numberOfSudokus);
TestSolver(cpSolverSimple, SudokuDifficulty.Hard, "CP Solver Simple", numberOfSudokus);
TestSolver(cpSolverMinSize, SudokuDifficulty.Hard, "CP Solver Min Size", numberOfSudokus);
TestSolver(mipSolver, SudokuDifficulty.Hard, "MIP Solver", numberOfSudokus);
TestSolver(satSolver, SudokuDifficulty.Hard, "SAT Solver", numberOfSudokus);

// Préparer les données pour le graphique Plotly
var solverNames = results.Select(r => r.SolverName).Distinct().ToArray();
var difficulties = results.Select(r => r.Difficulty).Distinct().ToArray();
var traces = new List<Trace>();

foreach (var solverName in solverNames)
{
    var solverResults = results.Where(r => r.SolverName == solverName).ToList();
    var trace = new Bar
    {
        x = solverResults.Select(r => r.Difficulty).ToArray(),
        y = solverResults.Select(r => r.Time).ToArray(),
        name = solverName
    };
    traces.Add(trace);
}

var layout = new Layout.Layout
{
    barmode = "group",
    title = "Comparison of Solver Performance by Difficulty",
    xaxis = new Xaxis { title = "Difficulty" },
    yaxis = new Yaxis { title = "Total Time (ms)" }
};

var chart = Chart.Plot(traces);
chart.WithLayout(layout);
display(chart);

<!DOCTYPE html>